In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# Simulation

In [9]:
# costs
costs = 0
T = 10000000
# initial inventory 
inventory = {"p1": 5, "p2": 5}
prods = ["p1", "p2"]
ph_costs = [1, 2]

# simulate for large T 
for t in range(T):
    #
    for idx, product in enumerate(prods):
        costs += inventory[product]*ph_costs[idx]

    # demand for individual products 
    demand = np.random.randint(0,2, 2)
    order = {"p1": 0, "p2": 0}
    hold_costs = 0

    # check how much to order 
    if inventory[prods[0]] == 1 or inventory[prods[1]] == 1:
        order[prods[0]] = 5 - inventory[prods[0]]
        order[prods[1]] = 5 - inventory[prods[1]]
        # order costs 
        costs += 5
    
    # update inventory level based on demand
    for idx, product in enumerate(prods):
          inventory[product] -= demand[idx]

    # update stock for next time step 
    for product in prods:
        inventory[product] += order[product]

   
print(f"Long-run avg. costs: {costs/T}")

Long-run avg. costs: 10.4458429


# Limiting Distribution

In [3]:
holding_cost_1, holding_cost_2 = 1, 2
state_space = []
num_states = 5
for i in range(1,num_states+1):
    for j in range(1,num_states+1):
        state_space.append((i, j))

# create transition matrix
P = np.zeros((len(state_space), len(state_space)))
# Create a dictionary to map states to indices - (1,1) - 0 and (5,5) - 24
state_to_index = {state: idx for idx, state in enumerate(state_space)}


# create the P-table 
for i, j in state_space:
    current_index = state_to_index[(i, j)]
    
    if i == 1 or j == 1:
        i = 5
        j = 5
    for d1 in [0, 1]:
        for d2 in [0, 1]:
            new_i = i - d1
            new_j = j - d2
            next_index = state_to_index[(new_i, new_j)]
            P[current_index, next_index] += 0.25

# Calculate the limiting distribution
distribution = np.zeros(len(state_space))

# start in state 25 - thus (5,5)
current_state = state_to_index[(5, 5)]
distribution[current_state] = 1

for t in range(10000):
    distribution = np.dot(distribution, P)

# Calculate the expected long run cost 
expected_cost = 0
ordering_cost = 5  # Assume the replenishment cost is 5, as per the simulation
c = np.zeros(len(state_space))

for i, j in state_space:
    idx = state_to_index[(i, j)]
    
    # Holding cost for the current state
    holding_cost = (holding_cost_1 * i + holding_cost_2 * j) 
    
    # Order cost: occurs if i == 1 or j == 1
    if i == 1 or j == 1:
        order_cost = ordering_cost 
    else:
        order_cost = 0
    total_cost = (holding_cost + order_cost) * distribution[idx]
    expected_cost += total_cost 
    # Per stage cost 
    c[idx] = holding_cost + order_cost
    
print(f"The expected cost is {expected_cost}")

The expected cost is 10.445065045248867


# Poisson equation

In [4]:
# number of states 
N = len(state_space)
transition_matrix = P
t = 100

# Needed to determine when it converges 
epsilon = 1e-6

# Reward are the cost per stage calculated earlier 
rewards = c

# Initialize the value function with 0 
V = np.zeros(N)

for i in range(t):
    
    V_next = rewards + np.dot(transition_matrix,V) 
    
    
    # Calculate the span of the difference
    span = np.max(V_next - V) - np.min(V_next - V)
    if span < epsilon:
        print(f"Converged after {i} iterations.")
        break
    
    # Normalize the value function
    # Update V for the next iteration
    V = V_next
print(f"Value function (relative): {V_next}")
#print(f"Long-run average cost (phi*): {optimal_action}")

Converged after 60 iterations.
Value function (relative): [638.1499016  640.1499016  642.1499016  644.1499016  646.1499016
 639.1499016  633.22314779 635.24756319 639.92236841 645.48063682
 640.1499016  632.91422986 631.86822665 636.4192994  644.01401488
 641.1499016  634.81125729 632.60448458 635.70391725 643.45232414
 642.1499016  637.44359979 635.69302723 638.07372331 645.1499016 ]


In [5]:
cost = np.dot(V_next -V, distribution)
print(cost)

10.445065045248889


# Bellman equation

Set up and support functions to perfome value iteration 

In [17]:
# as 0 can be a state we need to extend the state space to 21 states
state_space_bellman = []
num_states = 21
for i in range(1,num_states):
    for j in range(1, num_states):
        state_space_bellman.append((i, j))

# need this to map the states to the indices in the value function later 
state_to_index_bellman = {state: idx for idx, state in enumerate(state_space_bellman)}

prob = 1/4

def total_cost(i, j, action_1, action_2):
    '''
    Calculate the r(x,a) for each action
    '''
    holding_cost = i + 2 * j
    if action_1 == 0 and action_2 == 0:
        order_cost = 0 
    else:
        order_cost = 5
    return  holding_cost + order_cost

def total_number_of_actions(i, j):
    # the plus one is needed to include the zero action
    return 20 - i + 1, 20 - j + 1

# support function for getting costs at each state 
def get_costs(state, action):
    costs = 0
    # compute holding costs 
    costs += state[0] + state[1]*2

    # calculate order costs
    if action[0] > 0 or action[1] > 0:
        costs += 5
    
    return costs 

def get_long_run_average(policy, initial_state, time):
    """Compute long-term running avergae for a given policy and initial state"""
    T = time
    costs = 0
    state = initial_state
    for _ in range(T):
        action = policy[state]
        costs += get_costs(state, action)
        # simulate trasnition to next state with demand 
        demand = np.random.randint(0,2, 2)
        next_state = (state[0] + action[0] - demand[0], state[1] + action[1] - demand[1])
        # print(f"Current state {state}, state costs {V[state_index[state]]}, action {action}, next stet {next_state}")
        state = next_state
    
    
    return costs/T

In [21]:
# Initialize V_space appropriately
V_space = np.zeros(len(state_space_bellman))  # Or use initial cost estimates

# Set convergence threshold
convergence_threshold = 1e-6
max_iterations = 100  # Or any other appropriate number
prob = 0.25  # Assuming equal probability for each demand scenario

optimal_policy = {}
counter = 0
delta_history = []
for t in range(max_iterations):
    counter += 1    
    V_new = np.zeros_like(V_space)  
    delta = 0  # Initialize delta for convergence check

    # save all the actions for each state
    action_takes = {}

    for i, j in state_space_bellman:

        current_index = state_to_index_bellman[(i, j)]

        # Calculate for each state all possible actions - including from not ordering to ordering
        actions_i, actions_j = total_number_of_actions(i, j)

        # Constraint the action space as we must avoid lost sales
        start_i = 1 if i == 1 else 0
        start_j = 1 if j == 1 else 0

        action_vector = []
        action_taken = []

        # Create all possible actions
        all_possible_actions = []
        for action_1 in range(start_i, actions_i):
            for action_2 in range(start_j, actions_j):
                all_possible_actions.append((action_1, action_2))

        for action_1, action_2 in all_possible_actions:
            reward = total_cost(i, j, action_1, action_2)
            sum_future_states = []

            for d1 in [0, 1]:
                for d2 in [0, 1]:
                    # Update to the possible next state
                    new_i = i + action_1 - d1
                    new_j = j + action_2 - d2

                    # Get the index of the new state
                    new_state = (new_i, new_j)
                   
                    new_index = state_to_index_bellman[new_state]
                    future_value = prob * V_space[new_index]
                    sum_future_states.append(future_value)

            # Sum all future states
            sum_all_actions = np.sum(sum_future_states)

            # Total expected cost for this action
            total_expected_cost = reward + sum_all_actions

            action_vector.append(total_expected_cost)
            action_taken.append((action_1, action_2))

        # Find the action with the minimal total expected cost
        min_cost = np.min(action_vector)
        min_action = action_taken[np.argmin(action_vector)]

        # Update V_new and optimal policy
        V_new[current_index] = min_cost
        optimal_policy[(i, j)] = min_action

        # Update delta for convergence checking
        V_difference = V_new - V_space
        delta = np.max(V_difference) - np.min(V_difference)
   
    if delta < convergence_threshold: 
        print(f"Converged after {counter} iterations.")
        break
  

    # Update V_space for the next iteration
    V_space = V_new.copy() 

Converged after 78 iterations.


In [22]:
optimal_policy

{(1, 1): (2, 2),
 (1, 2): (2, 1),
 (1, 3): (2, 0),
 (1, 4): (2, 0),
 (1, 5): (2, 0),
 (1, 6): (2, 0),
 (1, 7): (2, 0),
 (1, 8): (2, 0),
 (1, 9): (2, 0),
 (1, 10): (2, 0),
 (1, 11): (2, 0),
 (1, 12): (2, 0),
 (1, 13): (2, 0),
 (1, 14): (2, 0),
 (1, 15): (2, 0),
 (1, 16): (2, 0),
 (1, 17): (2, 0),
 (1, 18): (2, 0),
 (1, 19): (2, 0),
 (1, 20): (2, 0),
 (2, 1): (1, 2),
 (2, 2): (0, 0),
 (2, 3): (0, 0),
 (2, 4): (0, 0),
 (2, 5): (0, 0),
 (2, 6): (0, 0),
 (2, 7): (0, 0),
 (2, 8): (0, 0),
 (2, 9): (0, 0),
 (2, 10): (0, 0),
 (2, 11): (0, 0),
 (2, 12): (0, 0),
 (2, 13): (0, 0),
 (2, 14): (0, 0),
 (2, 15): (0, 0),
 (2, 16): (0, 0),
 (2, 17): (0, 0),
 (2, 18): (0, 0),
 (2, 19): (0, 0),
 (2, 20): (0, 0),
 (3, 1): (0, 2),
 (3, 2): (0, 0),
 (3, 3): (0, 0),
 (3, 4): (0, 0),
 (3, 5): (0, 0),
 (3, 6): (0, 0),
 (3, 7): (0, 0),
 (3, 8): (0, 0),
 (3, 9): (0, 0),
 (3, 10): (0, 0),
 (3, 11): (0, 0),
 (3, 12): (0, 0),
 (3, 13): (0, 0),
 (3, 14): (0, 0),
 (3, 15): (0, 0),
 (3, 16): (0, 0),
 (3, 17): (0, 0),
 

In [23]:
initial_states = [(1, 1), (5, 5), (10, 10), (15, 15), (20, 20)]
T = 1000000
for state in initial_states:
    print(f"Initial state {state} , long-term average: {get_long_run_average(optimal_policy, state, T)}")

Initial state (1, 1) , long-term average: 7.98691
Initial state (5, 5) , long-term average: 7.985525
Initial state (10, 10) , long-term average: 7.988517
Initial state (15, 15) , long-term average: 7.989294
Initial state (20, 20) , long-term average: 7.988867


In [24]:
print(V_space / number_iterations)

[ 7.88105769  7.90669872  7.93233974  8.01577991  8.15071581  8.33693376
  8.57443376  8.86321581  9.20327991  9.59462607 10.03725427 10.53116453
 11.07635684 11.6728312  12.32058761 13.01962607 13.76994658 14.57154914
 15.42443376 16.32860041  7.89387821  7.85990385  7.89986111  7.98372863
  8.11866453  8.30488248  8.54238248  8.83116453  9.17122863  9.56257479
 10.00520299 10.49911325 11.04430556 11.64077991 12.28853632 12.98757479
 13.7378953  14.53949786 15.39238247 16.29654912  7.90669872  7.86994658
  7.89387821  7.97731838  8.11225427  8.29847222  8.53597222  8.82475427
  9.16481838  9.55616453  9.99879274 10.49270299 11.0378953  11.63436966
 12.28212607 12.98116453 13.73148504 14.53308761 15.38597222 16.29013887
  7.93672009  7.9046688   7.92389957  8.00028846  8.13273148  8.31811847
  8.55534148  8.84403121  9.18406453  9.57540043 10.01802521 10.51193433
 11.05712626 11.65360049 12.30135686 13.00039531 13.75071581 14.55231838
 15.40520299 16.30936964  7.99473291  7.96268162  7